<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [2]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [3]:
import datetime as dt
import logging
trial_ = dt.datetime.now().strftime('%Y_%M_%d_%H_%M')
logging.basicConfig(filename=f"results/ddpg/example_{trial_}.log",  level=logging.DEBUG)

import torch as th


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

import sys
sys.path.append("../finrl")

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from env_stocktrading_pair_trading_RATIO import StockPairTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import itertools

c:\Users\Administrador\AppData\Local\Programs\Python\Python38\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [6]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [7]:
TRAIN_START_DATE = '2010-10-19'
TRAIN_END_DATE = '2020-09-11' # 70% of All data set size (2556 aprox.)
TRADE_START_DATE = '2020-09-11'
TRADE_END_DATE = '2022-10-18'
PAIR_TRADE = ['AON','MMC']
# PAIR_TRADE = ['ACN','NDAQ']

In [8]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = PAIR_TRADE).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (6040, 8)


In [9]:
# print(config_tickers.DOW_30_TICKER)

In [10]:
df.shape

(6040, 8)

In [11]:
df.sort_values(['date','tic'],ignore_index=True).head()
# Structure : date (%Y-%m-%d), open, high, low, close, volume, tic, day

,date,open,high,low,close,volume,tic,day
0,2010-10-19,39.430000,39.910000,39.180000,34.570557,2891000,AON,1
1,2010-10-19,24.090000,24.540001,24.059999,19.009436,4187500,MMC,1
2,2010-10-20,39.590000,39.709999,39.419998,34.710629,2148600,AON,2
3,2010-10-20,24.440001,24.799999,24.340000,19.297817,2942600,MMC,2
4,2010-10-21,39.779999,40.360001,39.590000,35.069553,2539100,AON,3


In [12]:
df = df.pivot(index=['date','day'], columns=['tic'], values=['close']).reset_index(drop=False)
df.columns = ['date','day']+PAIR_TRADE
df['close'] = df[PAIR_TRADE[0]] / df[PAIR_TRADE[1]]
df['tic'] = '_'.join(PAIR_TRADE)
df = df[['date','tic','day','close']+PAIR_TRADE]

In [13]:
df.head()

,date,tic,day,close,AON,MMC
0,2010-10-19,AON_MMC,1,1.818600,34.570557,19.009436
1,2010-10-20,AON_MMC,2,1.798682,34.710629,19.297817
2,2010-10-21,AON_MMC,3,1.789101,35.069553,19.601774
3,2010-10-22,AON_MMC,4,1.784476,35.034531,19.632948
4,2010-10-25,AON_MMC,0,1.801968,35.279652,19.578396


# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [14]:
INDICATORS = ['close_5_sma']
# INDICATORS = []
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [15]:
# Removing trend and indicator used to remove it
processed['close'] /= processed[INDICATORS[0]]
processed.drop(INDICATORS, axis=1, inplace=True)
INDICATORS = []

In [16]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [17]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,day,close,AON,MMC
0,2010-10-19,AON_MMC,1.0,1.000000,34.570557,19.009436
1,2010-10-20,AON_MMC,2.0,0.994494,34.710629,19.297817
2,2010-10-21,AON_MMC,3.0,0.992772,35.069553,19.601774
3,2010-10-22,AON_MMC,4.0,0.992636,35.034531,19.632948
4,2010-10-25,AON_MMC,0.0,1.001892,35.279652,19.578396
5,2010-10-26,AON_MMC,1.0,1.011776,35.349689,19.422512
6,2010-10-27,AON_MMC,2.0,1.006907,35.778656,19.718685
7,2010-10-28,AON_MMC,3.0,1.004486,35.725929,19.679714
8,2010-10-29,AON_MMC,4.0,0.991583,34.926365,19.469280
9,2010-11-01,AON_MMC,0.0,0.987480,34.943943,19.601774


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [18]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

2491
529


In [19]:
train.tail()

,date,tic,day,close,AON,MMC
2486,2020-09-03,AON_MMC,3.0,0.997713,200.241821,112.768692
2487,2020-09-04,AON_MMC,4.0,1.002187,199.151489,111.636948
2488,2020-09-08,AON_MMC,1.0,0.994903,194.495361,109.770035
2489,2020-09-09,AON_MMC,2.0,0.995749,200.025726,112.933144
2490,2020-09-10,AON_MMC,3.0,0.999601,198.473679,111.830399


In [20]:
trade.head()

,date,tic,day,close,AON,MMC
0,2020-09-11,AON_MMC,4.0,0.996511,197.972717,111.994858
1,2020-09-14,AON_MMC,0.0,0.986236,198.385284,113.948807
2,2020-09-15,AON_MMC,1.0,0.984356,198.424561,114.742012
3,2020-09-16,AON_MMC,2.0,0.989747,198.945190,114.945129
4,2020-09-17,AON_MMC,3.0,0.997760,198.100418,114.026192


In [21]:
INDICATORS

[]

In [22]:
stock_dimension = len(train.tic.unique())
# Amount, Close_A, Close_B, Ratio, Position
state_space = 1 + 2*stock_dimension + 1 + 1 #len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 5


In [23]:
print('>> ',train.shape)
train.head()

>>  (2491, 6)


,date,tic,day,close,AON,MMC
0,2010-10-19,AON_MMC,1.0,1.000000,34.570557,19.009436
1,2010-10-20,AON_MMC,2.0,0.994494,34.710629,19.297817
2,2010-10-21,AON_MMC,3.0,0.992772,35.069553,19.601774
3,2010-10-22,AON_MMC,4.0,0.992636,35.034531,19.632948
4,2010-10-25,AON_MMC,0.0,1.001892,35.279652,19.578396


In [24]:
buy_cost_list = sell_cost_list = [0.001] * 2 #stock_dimension
num_stock_shares = [0] * stock_dimension

action_space = 3 # [-1:sell, 0:hold, 1:buy] # ==== OP1 
env_kwargs = {
    "hmax": 0,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": action_space, # ==== OP1 
    # "action_space":stock_dimension, # ==== OP2
    "reward_scaling": 1e-4,
    "print_verbosity" : 1,
    "model_name" : "test_pair_trading",
    "pair_name":'_'.join(PAIR_TRADE),
}

e_train_gym = StockPairTradingEnv(df = train, **env_kwargs)


> INITIAL STATUS : 	 [1000000, 34.570556640625, 19.009435653686523, 1.0, 0]


In [25]:
# plt.subplot(2,1,1)
# plt.plot(train['close'])
# # plt.subplot(2,1,2)
# # plt.plot(train['close']/train[INDICATORS[0]])
# plt.savefig(f"{RESULTS_DIR}/Close_RATIO__{trial_}.png")

## Environment for Training



In [26]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [27]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = True
if_using_ppo = False
if_using_td3 = False
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [28]:
# agent = DRLAgent(env = env_train)
# model_a2c = agent.get_model("a2c")

# if if_using_a2c:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/a2c'
#   new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_a2c.set_logger(new_logger_a2c)


In [29]:
# trained_a2c = agent.train_model(model=model_a2c, 
#                              tb_log_name='a2c',
#                              total_timesteps=50000) if if_using_a2c else None

In [30]:
env_train.observation_space

Box([-inf -inf -inf -inf -inf], [inf inf inf inf inf], (5,), float32)

### Agent 2: DDPG

In [31]:
DDPG_PARAMS = {"batch_size": 1000, "buffer_size": 300, "learning_rate": 0.05, "learning_starts" : 4000, 'tau':0.9, 'gradient_steps' : 700}
POLICY_KWARGS = {'net_arch' : [500,600,500,900,300]}


agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg", model_kwargs = DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 1000, 'buffer_size': 300, 'learning_rate': 0.05, 'learning_starts': 4000, 'tau': 0.9, 'gradient_steps': 700}
Using cpu device
>> 	 self.observation_space : 	 Box([-inf -inf -inf -inf -inf], [inf inf inf inf inf], (5,), float32)
>> self.obs_shape :  (5,)
Logging to results/ddpg


In [32]:
tmp_path

'results/ddpg'

In [33]:
trained_ddpg = agent.train_model(model=model_ddpg,
                                 tb_log_name='ddpg',
                                 total_timesteps=1500) if if_using_ddpg else None

self.state[0] :: 	 1000000 	actions[0] ::  -1.0
close_current_position from (S)	 :  False 	 PrevStatus :  -1.0
--- >> 	 self.state 1 :  [1000015.5075409947, 34.570556640625, 19.009435653686523, 1.0, -1.0]
** >> 	B_:1.0	 	S_:-1.0	 Assets: -1.0	 self.reward >>  0.0028180133945541457 

++ 	next_obs :  [[ 1.0000155e+06  3.4710629e+01  1.9297817e+01  9.9449360e-01
  -1.0000000e+00]]
**** >>> ADDING ACTION : 	 [[-0.9728935]] 	 for observation : 	 [[1.0000000e+06 3.4570557e+01 1.9009436e+01 1.0000000e+00 0.0000000e+00]]
self.state[0] :: 	 1000015.5075409947 	actions[0] ::  1.0
close_current_position from (B)	 :  True 	 PrevStatus :  0.0
--- >> 	 self.state 2 :  [1000000.0407212696, 34.710628509521484, 19.29781723022461, 0.9944935847160545, 0.0]
** >> 	B_:1.0	 	S_:-1.0	 Assets: 0.0	 self.reward >>  0.00021192824587924406 

++ 	next_obs :  [[1.00000006e+06 3.50695534e+01 1.96017742e+01 9.92771626e-01
  0.00000000e+00]]
**** >>> ADDING ACTION : 	 [[0.7020962]] 	 for observation : 	 [[ 1.0000155e

c:\Git\FinRL_cq\tutorials\1-Introduction\env_stocktrading_pair_trading_RATIO.py:295: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "index" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(


### Agent 3: PPO

In [34]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.01,
#     "learning_rate": 0.00025,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

# if if_using_ppo:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/a2c'
#   new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_ppo.set_logger(new_logger_ppo)

In [35]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [36]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

# if if_using_td3:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/td3'
#   new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_td3.set_logger(new_logger_td3)

In [37]:
# trained_td3 = agent.train_model(model=model_td3, 
#                              tb_log_name='td3',
#                              total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [38]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 5, # 100000,
#     "learning_rate": 0.001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

# if if_using_sac:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/sac'
#   new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_sac.set_logger(new_logger_sac)

In [39]:
# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=10) if if_using_sac else None

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [40]:
# data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
# insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [41]:
# insample_risk_indicator.vix.describe()

In [42]:
# insample_risk_indicator.vix.quantile(0.996)

In [43]:
# insample_risk_indicator.turbulence.describe()

In [44]:
# insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [45]:
# e_trade_gym = StockPairTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
e_trade_gym = StockPairTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

> INITIAL STATUS : 	 [1000000, 197.97271728515625, 111.99485778808594, 0.9965108184542988, 0]


In [46]:
trade.shape

(529, 6)

In [47]:
trained_moedl = trained_ddpg
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

TYPE : 	 <class 'stable_baselines3.td3.policies.TD3Policy'>
HERE>> 
self.state[0] :: 	 1000000 	actions[0] ::  -1.0
close_current_position from (S)	 :  False 	 PrevStatus :  -1.0
--- >> 	 self.state 1 :  [1000085.667891922, 197.97271728515625, 111.99485778808594, 0.9965108184542988, -1.0]
** >> 	B_:1.0	 	S_:-1.0	 Assets: -1.0	 self.reward >>  0.051950801302317995 

TYPE : 	 <class 'stable_baselines3.td3.policies.TD3Policy'>
HERE>> 
self.state[0] :: 	 1000085.667891922 	actions[0] ::  -1.0
close_current_position from (S)	 :  False 	 PrevStatus :  -2.0
--- >> 	 self.state 2 :  [1000169.792035492, 198.38528442382812, 113.94880676269531, 0.9862363220010741, -2.0]
** >> 	B_:1.0	 	S_:-1.0	 Assets: -2.0	 self.reward >>  0.024599101630854422 

TYPE : 	 <class 'stable_baselines3.td3.policies.TD3Policy'>
HERE>> 
self.state[0] :: 	 1000169.792035492 	actions[0] ::  -1.0
close_current_position from (S)	 :  False 	 PrevStatus :  -3.0
--- >> 	 self.state 3 :  [1000253.1614174424, 198.424560546875, 1

In [48]:
df_account_value['date'] = pd.to_datetime(df_account_value['date'], format='%Y-%m-%d')
df_account_value.shape

(529, 2)

In [49]:
plt.figure(figsize=(20,12))
plt.plot(df_account_value['date'],df_account_value['account_value'])
plt.savefig(f"{RESULTS_DIR}/ddpg/performance_Portfolio_trial_{trial_}.png")

In [50]:
plt.figure(figsize=(20,12))
plt.plot(train['date'],train['close'])
plt.savefig(f"{RESULTS_DIR}/ddpg/training_RATIO{trial_}.png")

In [51]:
plt.figure(figsize=(20,12))
plt.plot(trade['date'],trade['close'])
plt.savefig(f"{RESULTS_DIR}/ddpg/trading_RATIO_{trial_}.png")

In [52]:
trial_

'2022_09_13_17_09'

In [53]:
df_account_value.tail()

,date,account_value
524,2022-10-11,1.102921e+06
525,2022-10-12,1.102584e+06
526,2022-10-13,1.104986e+06
527,2022-10-14,1.102340e+06
528,2022-10-17,1.104577e+06


In [54]:
# df_actions#['L/S'].value_counts()
df_actions['actions'] = df_actions['actions'].explode()
df_actions['actions'].value_counts()

-1.0    528
Name: actions, dtype: int64

In [55]:
trained_ddpg.actor.mu#[0].weight.shape

Sequential(
  (0): Linear(in_features=5, out_features=500, bias=True)
  (1): ReLU()
  (2): Linear(in_features=500, out_features=600, bias=True)
  (3): ReLU()
  (4): Linear(in_features=600, out_features=500, bias=True)
  (5): ReLU()
  (6): Linear(in_features=500, out_features=900, bias=True)
  (7): ReLU()
  (8): Linear(in_features=900, out_features=300, bias=True)
  (9): ReLU()
  (10): Linear(in_features=300, out_features=1, bias=True)
  (11): Tanh()
)

In [56]:
trained_ddpg.critic.qf0

Sequential(
  (0): Linear(in_features=6, out_features=500, bias=True)
  (1): ReLU()
  (2): Linear(in_features=500, out_features=600, bias=True)
  (3): ReLU()
  (4): Linear(in_features=600, out_features=500, bias=True)
  (5): ReLU()
  (6): Linear(in_features=500, out_features=900, bias=True)
  (7): ReLU()
  (8): Linear(in_features=900, out_features=300, bias=True)
  (9): ReLU()
  (10): Linear(in_features=300, out_features=1, bias=True)
)

In [57]:
# amount, price_A, price_B, Qty_A, Qty_B, IndTec_A, IndTec_B
trained_ddpg.actor.mu(
    th.from_numpy(
        np.array([1000253.1614403534, 198.42457580566406, 114.74199676513672, 0.9843560092072803, -3.0])
    ).float()
)

tensor([-1.], grad_fn=<TanhBackward0>)

In [58]:
plt.hist(trained_ddpg.actor.mu[10].weight.detach().numpy().reshape(-1,1), bins=10)
plt.savefig(f"{RESULTS_DIR}/ddpg/actor_last_LAYER_weights_{trial_}.png")

In [59]:
trained_ddpg.actor.mu[10].weight.shape

torch.Size([1, 300])

In [60]:
raise Exception('asdfasdfasdf')

Exception: asdfasdfasdf

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])